In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import dates
%matplotlib inline
import statsmodels.api as sm
import seaborn as sns

In [ ]:
df_stores=pd.read_csv('/kaggle/input/retaildataset/stores data-set.csv')
df_features=pd.read_csv('/kaggle/input/retaildataset/Features data set.csv')
df_sales=pd.read_csv('/kaggle/input/retaildataset/sales data-set.csv')

In [ ]:
df_sales.head()

In [ ]:
df_features['Date'] = pd.to_datetime(df_features['Date'],format="%d/%m/%Y")
df_sales['Date'] = pd.to_datetime(df_sales['Date'],format="%d/%m/%Y")

****Common column between sales and store tables are store column lets connect on that****

In [ ]:
df1=df_stores.merge(df_sales,on='Store',how='right')
df1.sort_values("Date")

In [ ]:
df_features.head()

****Common tables on df1 and feature table are store, date and is holiday lets join on that****

In [ ]:
df=df1.merge(df_features,on=['Store','Date','IsHoliday'],how='left')

In [ ]:
df.info()

# Data Cleaning

In [ ]:
df.info()

We will not be working woth markdown so lets remove markdown

In [ ]:
df.columns

In [ ]:
df.drop(columns=['MarkDown1', 'MarkDown2', 'MarkDown3',
       'MarkDown4', 'MarkDown5'],inplace=True)

In [ ]:
df.info()

In [ ]:
df['Type'].value_counts()

In [ ]:
df[['Type','Dept','Size','Date']]=df[['Type','Dept','Size','Date']].fillna(method='ffill') #forward fill for categorical data
# if needed, here i don't need

In [ ]:
df.head()

Using KNN imputer method to fill the na value

In [ ]:
df.columns

In [ ]:
from sklearn.impute import KNNImputer
X = df[['Store', 'Size', 'Dept',
       'Temperature', 'Fuel_Price','CPI']].values
imputer = KNNImputer(n_neighbors=2)
df['CPI']=imputer.fit_transform(X)[:,-1] #if you got null value in CPI while joining or otherwise

In [ ]:
X = df[['Store', 'Size', 'Dept',
       'Temperature', 'Fuel_Price','Unemployment']].values
imputer = KNNImputer(n_neighbors=2)
df['Unemployment']=imputer.fit_transform(X)[:,-1]  #if you got null values in Unemployement while joining or otherwise

In [ ]:
#df.drop(df[df['Weekly_Sales'].isna()].index,inplace=True) # drop na of weekly sales

In [ ]:
df['IsHolidayNumeric']=df['IsHoliday'].apply(lambda x:1 if (str(x)=="True") else 0) # converted holiday true false to 0 and 1 respectively

# Time-Series Analysis

In [ ]:
df.describe()

In [ ]:
df2=df.groupby(by=['Date'], as_index=False)['Weekly_Sales'].sum()
f_1 = plt.figure(figsize=(12,6), dpi=100)
ax_1 = f_1.add_axes([0.0, 0.0, 0.9, 0.9])
ax_1.set_ylabel('Weekly_Sales')
ax_1.plot(df2['Date'], df2['Weekly_Sales'])
ax_1.grid(True, color='0.6', dashes=(5,2,1,2))
ax_1.set_facecolor((0.9, 0.9, 0.9))

#### Looks like from october to december the sales went up which we can tell that there could be event or sale on product at that point of time. As they mention holiday days lets check on that

In [ ]:
df2=df.groupby(by=['Date'], as_index=False)['Weekly_Sales'].sum()
f_1 = plt.figure(figsize=(12,6), dpi=100)
ax_1 = f_1.add_axes([0.0, 0.0, 0.9, 0.9])
ax_1.set_ylabel('Weekly_Sales')
ax_1.plot(df2['Date'], df2['Weekly_Sales'])

for x in df[df['IsHolidayNumeric']==1]['Date']:
    ax_1.axvline(x=x, color='g', linewidth=0.5)

#### we can see that in holidays there are peaks so we can say that people are more likely to go to stores on holidays or events that creates that holiday

In [ ]:
df2=df.groupby(by=['Date'], as_index=False)[['Weekly_Sales','Unemployment']].sum()
df2['Unemployment']=df2['Unemployment']*1000
f_1 = plt.figure(figsize=(12,6), dpi=100)
ax_1 = f_1.add_axes([0.0, 0.0, 0.9, 0.9])
ax_1.set_ylabel('Weekly_Sales')
ax_1.plot(df2['Date'], df2['Weekly_Sales'])
ax_1.plot(df2['Date'], df2['Unemployment'])
ax_1.legend(labels=['Weekly_Sales','Unemployment'])

#### Unemployment does not have a much difference on sales , we can safely say that there is no trend here

In [ ]:
df2=df.groupby(by=['Date'], as_index=False)[['Weekly_Sales','Size']].sum()
df2['Size']=df2['Size']*0.1
f_1 = plt.figure(figsize=(12,6), dpi=100)
ax_1 = f_1.add_axes([0.0, 0.0, 0.9, 0.9])
ax_1.set_ylabel('Weekly_Sales')
ax_1.plot(df2['Date'], df2['Weekly_Sales'])
ax_1.plot(df2['Date'], df2['Size'])
ax_1.legend(labels=['Weekly_Sales','Size'])

#### Company size also dosen't have a much of a difference in sales, we can safely say that there is no trend here

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

df2=df.groupby(by=['Date'], as_index=False)['Weekly_Sales'].sum()
df2=df2.set_index("Date")
#df2 = df2.asfreq('W')
#df2 = df2.fillna(method='bfill')
#df2.index=pd.DatetimeIndex(df2.index,freq=None)

res = seasonal_decompose(df2['Weekly_Sales'], model='mul')
plt.figure(figsize=(22,3))
res.seasonal.plot(title='Seasonal');
plt.figure(figsize=(22,3))
res.trend.plot(title='Trend');
plt.figure(figsize=(22,3))
res.resid.plot(title='Residual');

#### We can clearly see that there is some trend going on plus the data is seasonal which was clear from the first visualization but here is the confirmation that there is seasonality 

In [ ]:
df2=df.groupby(by=['Date'], as_index=False)[['Weekly_Sales','Size']].sum()
df1=df2.set_index("Date")
df1['Weekly_Sales'].plot(figsize=(20,6))
df1.rolling(window=6).mean()['Weekly_Sales'].plot()

### Further you can try analysis with Unemployement, Temperature, CPI

## Forecasting

In [ ]:
df1=df.groupby(by=['Date','IsHolidayNumeric'], as_index=False)['Weekly_Sales'].sum()
df1=df1.set_index("Date")

In [ ]:
train_df = df1['Weekly_Sales'].iloc[:120]
test_df = df1['Weekly_Sales'].iloc[120:]

## Holt-Winters Model

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing


fit_model = ExponentialSmoothing(train_df,
                                  seasonal='mul',
                                  seasonal_periods=52).fit()
prediction = fit_model.forecast(23)
prediction


test_df.plot(figsize=(12,6))
prediction.plot()

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
print("Mean Absolute Percentage Error = {a}%".format(a=mean_absolute_percentage_error(test_df,prediction)))

****We got error of just 1.762% that mean our model is fitted with good accuracy****

In [ ]:
def mse(y1, y2, axis=0):
    y1_np = y1.to_numpy()
    y2_np = y2.to_numpy()
    return ((y1_np - y2_np) ** 2).mean(axis=axis)

print("Root mean squared error = {a}".format(a=np.sqrt(mse(test_df, prediction, None))))

## Sarima with Exogenous variable (SARIMAX)

##### An **exogenous** variable is one whose value is determined outside the model and is imposed on the model. In other words, variables that affect a model without being affected by it.

In [ ]:
#!pip install pmdarima

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima
auto_arima(df1['Weekly_Sales'],exog=df1['IsHolidayNumeric'], seasonal=True, m=52, trace=True).summary() #auto arima gives the best fit

In [ ]:
train_df = df1.iloc[:120]
test_df = df1.iloc[120:]

In [ ]:
model = SARIMAX(train_df['Weekly_Sales'],exog=train_df['IsHolidayNumeric'], order=(2,0,2),
               seasonal_order=(1, 0, 0, 52))
res = model.fit()

In [ ]:
prediction=res.predict(start='2012-05-25',end="2012-10-26",exog=test_df['IsHolidayNumeric'])

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
print("Mean Absolute Percentage Error = {a}%".format(a=mean_absolute_percentage_error(test_df['Weekly_Sales'],prediction)))

**** Error percentage is more here than without exog variable so here exog variable have very less effect ****